In [1]:
## Load Packages

# system
from pathlib import Path
import sys

# external software
import yaml

# python internal
import pandas as pd
from datetime import date
from importlib.metadata import version
from datetime import date
from importlib.metadata import version

# Project Packages
from ir_pell_accepts.io_utils import infer_and_read_file
from ir_pell_accepts.paths import CONFIG_PATH
from ir_pell_accepts.headcount_calcs import grs_cohort_pell, grs_cohort, total_headcount, fall_enrollment
from ir_pell_accepts.clean import remove_leading_zeros
from ir_pell_accepts.helper import calc_percent, calc_academic_year_from_term, construct_cohort
from ir_pell_accepts.output import output_results, contruct_results_df


In [2]:
## Jupyter-Notebook Only -- comment-out when creating .py script

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_seq_items', 1000)


In [3]:
## Load Configuration File and store its values

# Check for config file
if not CONFIG_PATH.exists():
    raise FileNotFoundError(
        f"Config file not found at {CONFIG_PATH}. "
        "Create ir-<project>-<name>/configs/config.yaml to execute code"
    )

with CONFIG_PATH.open("r") as f:
    config = yaml.safe_load(f)

# File and folder paths
BOX_ROOT = Path(config["box_repo"]["root"]).expanduser()
DIR = Path(config["box_repo"]["pell_dir"]).expanduser()
PELL_PATH = DIR / Path(config["box_repo"]["pell_file"]).expanduser()
RETENTION_PATH = Path(config["box_repo"]["retention_dir"]).expanduser() / Path(config["box_repo"]["retention_file"]).expanduser()
ENROLLMENT_PATH = Path(config["box_repo"]["enrollment_dir"]).expanduser() / Path(config["box_repo"]["enrollment_file"]).expanduser()
RESULTS_PATH = Path(config["box_repo"]["results_dir"]) / Path(config["box_repo"]["results_file"])

# Project Parameters
term = config["params"]["term"] 
id_column = config["params"]["id_column"]


In [4]:
# Test configuation inputs
if not BOX_ROOT.exists():
    raise FileNotFoundError(f"Box repo path does not exist: {BOX_ROOT}")

if not DIR.exists():
    raise FileNotFoundError(f"path does not exist: {DIR}")

if not PELL_PATH.exists():
    raise FileNotFoundError(f"Input Pell file does not exist: {PELL_PATH}")

if not RETENTION_PATH.exists():
    raise FileNotFoundError(f"Input Retention file does not exist: {RETENTION_PATH}")

if not ENROLLMENT_PATH.exists():
    raise FileNotFoundError(f"Input Retention file does not exist: {ENROLLMENT_PATH}")

if not RESULTS_PATH.parent.exists():
    raise FileNotFoundError(f"Results path does not exist: {RESULTS_PATH.parent}")

if len(term) != 6:
    raise ValueError(f"Value for term, {term}, is invalid. Needs to be a 6 digit numeric. Ex: '202580'")


In [5]:
# Read in files (all columns coverted to strings)
df_pell = infer_and_read_file(PELL_PATH)
df_ret  = infer_and_read_file(RETENTION_PATH)
df_enrl = infer_and_read_file(ENROLLMENT_PATH)


In [6]:
# Standardize ID column
df_pell = remove_leading_zeros(df_pell, column=id_column)
df_ret  = remove_leading_zeros(df_ret, column=id_column)
df_enrl = remove_leading_zeros(df_enrl, column=id_column)


In [7]:
aid_year_column = 'AID_YEAR'
cohort_column = 'Cohort Name'
aid_year = calc_academic_year_from_term(term)
cohort = construct_cohort(term)
print(aid_year)
print(cohort)

2526
2025 Fall, First-Time, Full-Time


In [14]:
pids = df_pell.loc[df_pell[aid_year_column] == aid_year, id_column]
rids = df_ret.loc[df_ret[cohort_column] == cohort, id_column]


In [15]:
len(set(pids) & set(rids))

343

In [8]:
dfe = df_enrl
dfp = df_pell
dfr = df_ret

In [17]:
enrollment_conditions = (
        (dfe['Academic Period'] == term) &
        (dfe['Time Status'] == 'FT') &
        (dfe['Student Level'] == 'UG') &
        (dfe['Degree'] != 'Non Degree')
    )

In [18]:
incoming_transfer_cohort = term[0:4] + " " + "Fall, Transfer, Full-Time"


In [21]:
pids = dfp.loc[dfp['AID_YEAR'] == aid_year, id_column].dropna()
eids = dfe.loc[enrollment_conditions, id_column].dropna()
rids = dfr.loc[dfr['Cohort Name'] != incoming_transfer_cohort, id_column].dropna()
rids_t = dfr.loc[dfr['Cohort Name'] == incoming_transfer_cohort, id_column].dropna()


In [22]:
dfp_t = dfp[dfp['AID_YEAR'] == aid_year]


In [25]:
print(dfp_t.head())
print(dfp_t.shape)

              ID AID_YEAR
13699  100057265     2526
13700  101195564     2526
13701  101300108     2526
13702  101310537     2526
13703  101312434     2526
(1310, 2)


In [26]:
dfp_t2 = dfp_t.merge(dfr[['ID', 'Academic Period 1st Fall', 'Cohort Type Description']], on='ID', how='left')

In [27]:
dfp_t2.shape

(1310, 4)

In [29]:
dfp_t2.head()

,ID,AID_YEAR,Academic Period 1st Fall,Cohort Type Description
0,100057265,2526,202480,"Fall, First-Time, Full-Time"
1,101195564,2526,NaN,"Spring, Transfer, Full-Time"
2,101300108,2526,NaN,"Spring, Transfer, Full-Time"
3,101310537,2526,NaN,"Spring, Transfer, Part-Time"
4,101312434,2526,NaN,"Spring, Transfer, Full-Time"


In [30]:
dfp_t2['Academic Period 1st Fall'].value_counts()

Academic Period 1st Fall
202580    399
202480    300
202380    255
202280    245
202180     49
202080     12
201980      7
201780      2
Name: count, dtype: int64

In [43]:
dfe_t = dfe[enrollment_conditions]

In [44]:
dfe_t.shape

(6939, 35)

In [45]:
dfe_t2 = dfe_t.merge(dfr[['ID', 'Academic Period 1st Fall', 'Cohort Name']], on='ID', how='left')


In [46]:
dfe_t2.head()

,Academic Period,Address Start Date (most recent address),CIP,City (most recent address),College/School,College/School Code,County Code (most recent address),County Desc (most recent address),Degree,Department/Area,Dept/Area Code,Gender,ID,INDI in Academic Period Starting Fall 2025,Major,Major Code,Name,Nation (most recent address),Nation Desc (most recent address),Nation Of Citizenship Desc,Nation Of Citizenship Desc (susan's data),Permanent Residence State/Nation,Postal Code (most recent address),Raceethnicity,Religion,State Province (most recent address),State Province Desc (most recent address),Street Line1 (most recent address),Student Level,Time Status,UDSA in Academic Period (Starting Fall 2025),UG/LW Class,Course Billing Credits,Person Uid,Person Uid (UD Person),Academic Period 1st Fall,Cohort Name
0,202580,10/25/2021 12:00:00 AM,140801,Dover,School of Engineering,EN,39157,OH-Tuscarawas,Bachelor of Civil Engineering,Civil & Env Engr Engr Mech,CEE,Men,101718675,NaN,Civil Engineering,CEE,"Williams, Dawson",NaN,NaN,NaN,NaN,OH,44622-9617,White,Protestant,OH,Ohio,8933 Sandstone Rd NE,UG,FT,NaN,Senior,12,1925336,1925336,202280,"2022 Fall, First-Time, Full-Time"
1,202580,12/21/2022 12:00:00 AM,141001,New Carlisle,School of Engineering,EN,39023,OH-Clark,Bachelor of Electrical Engr.,Electrical & Computer Engr,ECE,Men,101717962,NaN,Electrical Engineering,ELE,"Locke, Noah D.",NaN,NaN,NaN,NaN,OH,45344-9571,White,Protestant,OH,Ohio,12330 Stafford Rd,UG,FT,NaN,Senior,12,1924612,1924612,202280,"2022 Fall, First-Time, Full-Time"
2,202580,10/27/2021 12:00:00 AM,141001,Miamisburg,School of Engineering,EN,39113,OH-Montgomery,Bachelor of Electrical Engr.,Electrical & Computer Engr,ECE,Men,101713201,NaN,Electrical Engineering,ELE,"Wells, Matthew M.",NaN,NaN,NaN,NaN,OH,45342-1456,White,Protestant,OH,Ohio,5281 Soldiers Home Msbg Rd,UG,FT,NaN,Senior,12,1919831,1919831,202280,"2022 Fall, First-Time, Full-Time"
3,202580,11/30/2021 12:00:00 AM,141901,Saint Louis,School of Engineering,EN,29510,MO-Saint Louis City,Bachelor of Mechanical Engr.,Mechanical Engineering,MAE,Men,101719002,NaN,Mechanical Engineering,MEE,"Murphy, Elijah M.",NaN,NaN,NaN,NaN,MO,63109-3536,White,Protestant,MO,Missouri,4945 Finkman St,UG,FT,NaN,Senior,12,1925664,1925664,202280,"2022 Fall, First-Time, Full-Time"
4,202580,10/25/2020 12:00:00 AM,141901,Liberty Township,School of Engineering,EN,39017,OH-Butler,Bachelor of Mechanical Engr.,Mechanical Engineering,MAE,Men,101686551,NaN,Mechanical Engineering,MEE,"Collett, Kyle S.",NaN,NaN,NaN,NaN,OH,45044-5779,White,Protestant,OH,Ohio,5888 Alder Ct,UG,FT,NaN,Senior,12,1893043,1893043,202180,"2021 Fall, First-Time, Full-Time"


In [47]:
sum(dfe_t2['Cohort Name'].isna())


12

In [48]:
dfe_t2b = dfe_t2[dfe_t2['Cohort Name'].isna()]

In [49]:
dfe_t2b

,Academic Period,Address Start Date (most recent address),CIP,City (most recent address),College/School,College/School Code,County Code (most recent address),County Desc (most recent address),Degree,Department/Area,Dept/Area Code,Gender,ID,INDI in Academic Period Starting Fall 2025,Major,Major Code,Name,Nation (most recent address),Nation Desc (most recent address),Nation Of Citizenship Desc,Nation Of Citizenship Desc (susan's data),Permanent Residence State/Nation,Postal Code (most recent address),Raceethnicity,Religion,State Province (most recent address),State Province Desc (most recent address),Street Line1 (most recent address),Student Level,Time Status,UDSA in Academic Period (Starting Fall 2025),UG/LW Class,Course Billing Credits,Person Uid,Person Uid (UD Person),Academic Period 1st Fall,Cohort Name
323,202580,10/6/2021 12:00:00 AM,150805,Dayton,School of Engineering,EN,39113,OH-Montgomery,BS in Engineering Technology,Engineering Technology,EGT,Men,101713264,NaN,Mechanical Engineering Tech,MCT,"Joseph, Alexander M.",NaN,NaN,NaN,NaN,OH,45405-4214,Two or More Races,Unknown,OH,Ohio,1100 N Main St,UG,FT,NaN,Senior,18,1919894,1919894,NaN,NaN
2394,202580,10/25/2020 12:00:00 AM,230101,Oakwood,College of Arts & Sciences,AS,39113,OH-Montgomery,Bachelor of Arts,English,ENG,Men,101688785,NaN,English,ENG,"Hamid, Shiraz R.",NaN,NaN,NaN,NaN,OH,45419-3330,Asian,Other,OH,Ohio,28 Thruston Blvd W,UG,FT,NaN,Junior,12,1895282,1895282,NaN,NaN
4389,202580,1/12/2022 12:00:00 AM,110101,Dayton,College of Arts & Sciences,AS,39113,OH-Montgomery,Bachelor of Science,Computer Science,CPS,Men,101704893,NaN,Computer Information Systems,CIS,"Rubuto, Gislain G.",NaN,NaN,NaN,NaN,OH,45404-2241,Black or African American,Unknown,OH,Ohio,407 Brandt St,UG,FT,NaN,Senior,18,1911447,1911447,NaN,NaN
4521,202580,7/2/1925 12:00:00 AM,380101,Bangalore,College of Arts & Sciences,AS,NaN,NaN,Bachelor of Arts,Philosophy,PHL,Men,101892431,India Students (Marianists),Philosophy,PHL,"Soreng, Ajay",IN,India,NaN,NaN,India,56009,Unknown,NaN,NaN,NaN,Deepahalli College,UG,FT,NaN,Freshman,19,2099703,2099703,NaN,NaN
4522,202580,7/2/2025 12:00:00 AM,380101,Bangalore,College of Arts & Sciences,AS,NaN,NaN,Bachelor of Arts,Philosophy,PHL,Men,101892432,India Students (Marianists),Philosophy,PHL,"Mariaya, Rahul",IN,India,NaN,NaN,India,560099,Unknown,NaN,NaN,NaN,Deepahalli College,UG,FT,NaN,Freshman,19,2099704,2099704,NaN,NaN
4523,202580,7/2/2025 12:00:00 AM,380101,Bangalore,College of Arts & Sciences,AS,NaN,NaN,Bachelor of Arts,Philosophy,PHL,Men,101892433,India Students (Marianists),Philosophy,PHL,"Xalxo, Tej Kumar",IN,India,NaN,NaN,India,560099,Unknown,NaN,NaN,NaN,Deepahalli College,UG,FT,NaN,Freshman,19,2099705,2099705,NaN,NaN
4524,202580,7/2/2025 12:00:00 AM,380101,Bangalore,College of Arts & Sciences,AS,NaN,NaN,Bachelor of Arts,Philosophy,PHL,Men,101892434,India Students (Marianists),Philosophy,PHL,"Kujur, Wilfred",IN,India,NaN,NaN,India,560099,Unknown,NaN,NaN,NaN,Deepahalli College,UG,FT,NaN,Freshman,19,2099706,2099706,NaN,NaN
4525,202580,8/4/2022 12:00:00 AM,380101,Bangalore,College of Arts & Sciences,AS,NaN,NaN,Bachelor of Arts,Philosophy,PHL,Men,101750217,India Students (Marianists),Philosophy,PHL,"Kiran Kumar, Chilka",IN,India,NaN,NaN,India,56099,Unknown,NaN,NaN,NaN,"Deepahalli, Post Bag 1 Ghandpura",UG,FT,NaN,Senior,13,1956981,1956981,NaN,NaN
5404,202580,9/5/2021 12:00:00 AM,150805,Englewood,School of Engineering,EN,39113,OH-Montgomery,BS in Engineering Technology,Engineering Technology,EGT,Men,101714314,NaN,Mechanical Engineering Tech,MCT,"Hartman, Keaton R.",NaN,NaN,NaN,NaN,OH,45322-1305,Hispanic,Roman Catholic,OH,Ohio,624 N Main St,UG,FT,NaN,Senior,18,1920952,1920952,NaN,NaN
5449,202580,4/7/2025 2:34:28 PM,141901,Dayton,School of Engineering,EN,39113,OH-Montgomery,Bachelor of Mechanical Engr.,Mechanical Engineering,MAE,Men,3940724,NaN,Mechanical Engineering,MEE,"Brailer, Evan G.",US,United States,NaN,NaN,OH,45402-1655,White,Unknown,OH,Ohio,340 E 1st St Unit 630,UG,FT,NaN,

In [58]:
dfr[dfr['ID'].isin(dfe_t2b['ID'])]

,Academic Period 1st Fall,Academic Period 1st Spring,Academic Period 2nd Fall,Academic Period 2nd Spring,Academic Period 3rd Fall,Academic Period 3rd Spring,Academic Period 4th Fall,Academic Period 4th Spring,Academic Period 5th Fall,Academic Period 6th Fall,Academic Period 6th Spring,Academic Period 7th Fall,Academic Period 8th Fall,Academic Period 9th Fall,Academic Period 9th Spring,Academic Period Graduation,Academic Period5th Spring,Academic Period7th Spring,Academic Period8th Spring,Academic Standing Desc 1st Fall,Academic Standing Desc 1st Spring,Academic Standing Desc 2nd Fall,Academic Standing Desc 2nd Spring,Academic Standing Desc 3rd Fall,Academic Standing Desc 3rd Spring,Academic Standing Desc 4th Fall,Academic Standing Desc 4th Spring,Academic Standing Desc 5th Fall,Academic Standing Desc 6th Fall,Academic Standing Desc 6th Spring,Academic Standing Desc 7th Fall,Academic Standing Desc 8th Fall,Academic Standing Desc 9th Fall,Academic Standing Desc 9th Spring,Academic Standing Desc5th Spring,Academic Standing Desc7th Spring,Academic Standing Desc8th Spring,Cohort,Cohort Academic Period,Cohort Desc,Cohort Fiscal Year,Cohort Name,Cohort Season,Cohort Start Date (for IPEDS Graduation Versions),Cohort Start Term Name,Cohort Start Terms,Cohort Time Status,Cohort Transfer Status,Cohort Type,Cohort Type Description,...,Retained to or graduated before 4th year from 3rd year (excludes students not retained to 3rd year),Retained to or graduated before 4th Year Spring,Retained to or graduated before 5th Year,Retained to or graduated before 5th Year Spring,Retained to or graduated before 6th Year,Retained to or graduated before 6th Year Spring,Retained to or graduated before 7th Year,Retained to or graduated before 7th Year Spring,Retained to or graduated before 8th Year,Spring Adjusted to Years to Graduation for Retention Measure,Term Updated for Graduation,Term Updated for Graduation - Cohort Start Term,Trans Credit Earned Cumulative 1st Fall,Trans Credit Earned Cumulative 1st Spring,Trans Credit Earned Cumulative 2nd Fall,Trans Credit Earned Cumulative 2nd Spring,Trans Credit Earned Cumulative 3rd Fall,Trans Credit Earned Cumulative 3rd Spring,Trans Credit Earned Cumulative 4th Fall,Trans Credit Earned Cumulative 4th Spring,Trans Credit Earned Cumulative 5th Fall,Trans Credit Earned Cumulative 6th Fall,Trans Credit Earned Cumulative 6th Spring,Trans Credit Earned Cumulative 7th Fall,Trans Credit Earned Cumulative 8th Fall,Trans Credit Earned Cumulative 9th Fall,Trans Credit Earned Cumulative 9th Spring,Trans Credit Earned Cumulative5th Spring,Trans Credit Earned Cumulative7th Spring,Trans Credit Earned Cumulative8th Spring,University Honors Program,University Honors Program 1st Fall,University Honors Program 1st Spring,University Honors Program 2nd Fall,University Honors Program 2nd Spring,University Honors Program 3rd Fall,University Honors Program 3rd Spring,University Honors Program 4th Fall,University Honors Program 4th Spring,University Honors Program 5th Fall,University Honors Program 6th Fall,University Honors Program 6th Spring,University Honors Program 7th Fall,University Honors Program 8th Fall,University Honors Program 9th Fall,University Honors Program 9th Spring,University Honors Program5th Spring,University Honors Program7th Spring,University Honors Program8th Spring,Years to Graduation


In [59]:
dfp[dfp['ID'].isin(dfe_t2b['ID'])]

,ID,AID_YEAR
10414,101704893,2223
11470,101704893,2324
12592,101704893,2425
12622,101713264,2425
12629,101714314,2425
13787,101704893,2526
13798,101713264,2526
13805,101714314,2526


In [60]:
dfr[dfr['Person Uid']=="1925664"]

,Academic Period 1st Fall,Academic Period 1st Spring,Academic Period 2nd Fall,Academic Period 2nd Spring,Academic Period 3rd Fall,Academic Period 3rd Spring,Academic Period 4th Fall,Academic Period 4th Spring,Academic Period 5th Fall,Academic Period 6th Fall,Academic Period 6th Spring,Academic Period 7th Fall,Academic Period 8th Fall,Academic Period 9th Fall,Academic Period 9th Spring,Academic Period Graduation,Academic Period5th Spring,Academic Period7th Spring,Academic Period8th Spring,Academic Standing Desc 1st Fall,Academic Standing Desc 1st Spring,Academic Standing Desc 2nd Fall,Academic Standing Desc 2nd Spring,Academic Standing Desc 3rd Fall,Academic Standing Desc 3rd Spring,Academic Standing Desc 4th Fall,Academic Standing Desc 4th Spring,Academic Standing Desc 5th Fall,Academic Standing Desc 6th Fall,Academic Standing Desc 6th Spring,Academic Standing Desc 7th Fall,Academic Standing Desc 8th Fall,Academic Standing Desc 9th Fall,Academic Standing Desc 9th Spring,Academic Standing Desc5th Spring,Academic Standing Desc7th Spring,Academic Standing Desc8th Spring,Cohort,Cohort Academic Period,Cohort Desc,Cohort Fiscal Year,Cohort Name,Cohort Season,Cohort Start Date (for IPEDS Graduation Versions),Cohort Start Term Name,Cohort Start Terms,Cohort Time Status,Cohort Transfer Status,Cohort Type,Cohort Type Description,...,Retained to or graduated before 4th year from 3rd year (excludes students not retained to 3rd year),Retained to or graduated before 4th Year Spring,Retained to or graduated before 5th Year,Retained to or graduated before 5th Year Spring,Retained to or graduated before 6th Year,Retained to or graduated before 6th Year Spring,Retained to or graduated before 7th Year,Retained to or graduated before 7th Year Spring,Retained to or graduated before 8th Year,Spring Adjusted to Years to Graduation for Retention Measure,Term Updated for Graduation,Term Updated for Graduation - Cohort Start Term,Trans Credit Earned Cumulative 1st Fall,Trans Credit Earned Cumulative 1st Spring,Trans Credit Earned Cumulative 2nd Fall,Trans Credit Earned Cumulative 2nd Spring,Trans Credit Earned Cumulative 3rd Fall,Trans Credit Earned Cumulative 3rd Spring,Trans Credit Earned Cumulative 4th Fall,Trans Credit Earned Cumulative 4th Spring,Trans Credit Earned Cumulative 5th Fall,Trans Credit Earned Cumulative 6th Fall,Trans Credit Earned Cumulative 6th Spring,Trans Credit Earned Cumulative 7th Fall,Trans Credit Earned Cumulative 8th Fall,Trans Credit Earned Cumulative 9th Fall,Trans Credit Earned Cumulative 9th Spring,Trans Credit Earned Cumulative5th Spring,Trans Credit Earned Cumulative7th Spring,Trans Credit Earned Cumulative8th Spring,University Honors Program,University Honors Program 1st Fall,University Honors Program 1st Spring,University Honors Program 2nd Fall,University Honors Program 2nd Spring,University Honors Program 3rd Fall,University Honors Program 3rd Spring,University Honors Program 4th Fall,University Honors Program 4th Spring,University Honors Program 5th Fall,University Honors Program 6th Fall,University Honors Program 6th Spring,University Honors Program 7th Fall,University Honors Program 8th Fall,University Honors Program 9th Fall,University Honors Program 9th Spring,University Honors Program5th Spring,University Honors Program7th Spring,University Honors Program8th Spring,Years to Graduation
23585,202280,202310,202380,202410,202480,202510,202580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Calculated,Good Standing,Good Standing,Good Standing,Good Standing,Good Standing,Good Standing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22FED,202280,2022 Fall 1st Year Full-Time,2023,"2022 Fall, First-Time, Full-Time",Fall,8/31/2022,Fall 2022,202280,NaN,First-Time,FED,"Fall, First-Time, Full-Time",...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202553,273,18,39,39,39,39,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
enrollment_conditions = (
        (dfe['Academic Period'] == term) &
        (dfe['Time Status'] == 'FT') &
        (dfe['Student Level'] == 'UG') &
        (dfe['Degree'] != 'Non Degree')
    )

aid_year = calc_academic_year_from_term(term) 
incoming_transfer_cohort = term[0:4] + " " + "Fall, Transfer, Full-Time"

pids = dfp.loc[dfp['AID_YEAR'] == aid_year, id_column].dropna()
eids = dfe.loc[enrollment_conditions, id_column].dropna()
rids = dfr.loc[dfr['Cohort Name'] != incoming_transfer_cohort, id_column].dropna()
rids_t = dfr.loc[dfr['Cohort Name'] == incoming_transfer_cohort, id_column].dropna()
    

In [11]:
len(set(eids))

6939

In [12]:
len(set(eids) & set(rids))

6764

In [13]:
len(set(rids))

28841

In [14]:
len(set(rids_t))


163

In [15]:
len(set(eids)) - len(set(rids_t))

6776

In [18]:
len(set(pids) & set(eids)) - len(set(pids) & set(eids) & set(rids_t))

1217

In [20]:
len(set(rids_t) & set(pids))

54

In [21]:
len(set(rids) & set(pids))

1253

In [22]:
len(set(pids)) - len(set(pids) & set(eids) & set(rids_t))

1256

In [23]:
n_pell_intr = len(set(pids) & set(eids) & set(rids_t))

In [24]:
n_pell_intr


54

In [28]:
len(set(pids) & set(eids)) - n_pell_intr

1217

In [27]:
len(set(pids) & set(eids))

1271